### Obtener los links de donde vamos a sacar la información para la base de conocimiento

In [1]:
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import hashlib
import pickle

In [2]:
# get links from news page
URL = "https://www.metrocuadrado.com/noticias/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
links_list = soup.find_all("a", class_="notice-item")
links_list = [
    URL + x["href"].replace("/noticias/", "") 
    for x in links_list
]

In [3]:
print(f"Total links a scrapear: {len(links_list)}")
links_list[:2]

Total links a scrapear: 15


['https://www.metrocuadrado.com/noticias/actualidad/baja-de-tasas-de-interes-facilita-la-compra-de-vivienda-en-2024-4799',
 'https://www.metrocuadrado.com/noticias/actualidad/la-app-de-metrocuadrado-se-renueva-para-encontrar-inmuebles-mas-facil-4774']

### Obtener el contenido de cada link

In [4]:
def clean_text(x: str):

    x_clean = x.replace("\xa0", " ")
    x_clean = x_clean.replace("¿", "")
    x_clean = x_clean.strip()
    x_clean = x_clean.lower()
    x_clean = unidecode(x_clean)
    if x_clean.startswith("te puede interesar: "):
        x_clean = ""
    if x_clean.startswith("por: "):
        x_clean = ""

    return x_clean

In [5]:
docs_list = []

for URL in links_list:
    print(f"Scraping: {URL}")
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    # title content
    ele_h1 = soup.find("h1").text
    ele_h1 = clean_text(ele_h1)

    # generacion del id del documento
    hash_object = hashlib.md5(ele_h1.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[-8:]

    # body content
    elements = []
    for ele in soup.find_all("div", class_="article-text")[0].find_all("p"):
        ele_p = ele.text
        if ele_p not in ["", "\xa0"]:
            ele_p = clean_text(ele_p)
            elements.append(ele_p)

    elements = "\n".join(elements)
    docs_list.append(
        {
            "id": document_id,
            "url": URL,
            "title": ele_h1, 
            "content": elements
        }
    )

Scraping: https://www.metrocuadrado.com/noticias/actualidad/baja-de-tasas-de-interes-facilita-la-compra-de-vivienda-en-2024-4799


Scraping: https://www.metrocuadrado.com/noticias/actualidad/la-app-de-metrocuadrado-se-renueva-para-encontrar-inmuebles-mas-facil-4774
Scraping: https://www.metrocuadrado.com/noticias/actualidad/abc-de-la-nueva-ley-de-propiedad-horizontal-en-colombia-4770
Scraping: https://www.metrocuadrado.com/noticias/guia-de-compra/es-mejor-comprar-vivienda-para-vivir-o-invertir-4740
Scraping: https://www.metrocuadrado.com/noticias/actualidad/cuales-son-los-mejores-barrios-para-vivir-en-cali-4764
Scraping: https://www.metrocuadrado.com/noticias/guia-de-compra-y-venta/yumbo-un-destino-residencial-prometedor-en-el-valle-del-cauca-4773
Scraping: https://www.metrocuadrado.com/noticias/noticias-y-tendencias/como-se-calcula-la-ganancia-ocasional-por-la-venta-de-inmuebles-4757
Scraping: https://www.metrocuadrado.com/noticias/guia-de-compra/como-escoger-el-mejor-lugar-para-vivir-4737
Scraping: https://www.metrocuadrado.com/noticias/guia-de-compra/como-buscar-y-encontrar-inmuebles-4751
Scraping: https://www.

In [6]:
docs_list

[{'id': '2a3d5c52',
  'url': 'https://www.metrocuadrado.com/noticias/actualidad/baja-de-tasas-de-interes-facilita-la-compra-de-vivienda-en-2024-4799',
  'title': 'baja de tasas de interes facilita la compra de vivienda en 2024',
  'content': 'la compra de vivienda para muchos colombianos se ha visto afectada por diferentes factores economicos como la inflacion, devaluacion de la moneda, entre otros. frente a este panorama, muchos sectores, como el financiero, presentan medidas para la reactivacion de la economia.\nel banco de la republica viene haciendo reduccion de tasas de interes desde febrero del 2024, llegando en julio a 11.25%, y logrando a los mismos valores de octubre de 2022 cuando inicio un ascenso considerable de estas tasas.\nestas medidas, se hacen efectivas y positivas para los compradores de vivienda porque desde los primeros dias de julio, 10 entidades bancarias se han sumado y han bajado las tasas de interes para:\nlas 10 entidades bancarias que han entrado en la denom

In [7]:
print("# de documentos obtenidos:", len(docs_list))
print(
    "# de ids unicos construidos:",
    len(set([x["id"] for x in docs_list]))
)

# de documentos obtenidos: 15
# de ids unicos construidos: 15


In [8]:
# save file
with open('documents.bin', 'wb') as handle:
    pickle.dump(docs_list, handle)

----